# Employees Productivity Prediction using Random Forest Classifier
&emsp;This [Dataquest guided project](https://app.dataquest.io/c/140/m/755/guided-project%3A-predicting-employee-productivity-using-tree-models/1/introduction) uses a dataset from [UC Irvine Machine Learning Repository](https://archive.ics.uci.edu/dataset/597/productivity+prediction+of+garment+employees) to practise the application of Random Forest Classifier. Although instructions are given as it is a guided project from Dataquest, approach and methods for reaching the objective, predicting the productivity of employees, could be different.

1. [Dataset/Libraries Initialization](#initial)

# Dataset/Libraries Initialization<a id='initial'></a>

In [1]:
# array and dataframe
import numpy as np
import pandas as pd
# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use("fivethirtyeight")
%matplotlib inline
import seaborn as sns
# supervised machine learning algorithm and cross validation
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# metrics
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc

In [2]:
!pip install ucimlrepo
from ucimlrepo import fetch_ucirepo
# fetch dataset
productivity_prediction_of_garment_employees = fetch_ucirepo(id=597)
# data (as pandas dataframes)
X = productivity_prediction_of_garment_employees.data.features
y = productivity_prediction_of_garment_employees.data.targets
Xy = pd.concat([X, y], axis=1)


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [3]:
Xy.head(3)

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
